In [3]:
import pandas as pd
import json
import requests 
import time
import random
from urllib.parse import urlparse

In [4]:
#Define web url headers
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept':'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.9,vi;q=0.8,th;q=0.7,zh-CN;q=0.6,zh;q=0.5',
    'X-Guest-Token': 'Ae1PtIuUDbozkgpQ6VOW57jKqvchZxF2',
    'Connection' : 'keep-alive',
    'TE': 'Trailers'
}

In [ ]:

# list_url = ['https://tiki.vn/giay-the-thao-nam-co-cao/c49622',
#             'https://tiki.vn/giay-luoi-vai-nam/c49624',
#             'https://tiki.vn/giay-luoi-nhua-nam/c49628',
#             'https://tiki.vn/giay-tay-nam-co-day/c49630',
#             'https://tiki.vn/giay-sandals-nam-quai-ngang/c49634',
#             'https://tiki.vn/giay-sandals-nam-quai-cheo/c49636',
#             'https://tiki.vn/dep-nam-xo-ngon/c10383',
#             'https://tiki.vn/dep-nam-di-trong-nha/c49638',
#             'https://tiki.vn/giay-boots-nam/c8337',
#             'https://tiki.vn/ve-sinh-giay/c49644',
#             'https://tiki.vn/mieng-lot-giay-nam/c49646',
#             'https://tiki.vn/phu-kien-cho-giay-nam/c49648',
#             'https://tiki.vn/giay-the-thao-nam-co-thap/c49620',
#             'https://tiki.vn/giay-luoi-da-nam/c49626',
#             'https://tiki.vn/giay-tay-nam-khong-day/c49632',
#             'https://tiki.vn/dep-nam-quai-ngang/c10384'
#             ]

# Crawl Product ID

In [7]:
#create a list of URL which is all of the sub categories within a root category. Note that if a sub category already less than 50 pages
# then I will not go deeper into its' child categories anymore

list_url = ['https://tiki.vn/giay-the-thao-nam-co-cao/c49622',
            'https://tiki.vn/giay-tay-nam-khong-day/c49632',
            'https://tiki.vn/dep-nam-quai-ngang/c10384'
            ]

list_page_50 = []  #this list contains URL that have more than 50 pages
for url in list_url:    #get category name and category id using urlparse lib
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')

    category_name = path_segments[1]
    category_id = path_segments[2][1:]

    product_data = []  # this list will hold data of ID, category, maximum page within category
    page_error = []    # This list will hold data of error pages that we cannot request. So that we can crawl only 

    params = {                     # website url params, we will loop for different params, which means request different urls
        'limit':'40', 
        'include': 'advertisement',
        'aggregations': '2',
        'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
        'category': category_id,  # change based on category id
        'page': '1',              # I set page=1, later in the loop I will change page from 1 to last_page
        'urlKey': category_name   # change based on category name
    }
    
    response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)
    last_page = response.json().get('paging').get('last_page')  # I crawl last_page from TIKI, this is important to distinguish category with less than 50 pages and category >50 pages.

    print('total pages',last_page)
    print('CATEGORY ', category_name)

    if last_page == 50:           # TIKI API only allow to show 50 pages maximum. So if category with last_page = 50, we will make another list then try to use filters to divide them.  
        list_page_50.append(url)  # Append list_page_50 the url >50 pages
        continue 
    else:
        for i in range(1, last_page + 1):   # Crawl from page 1 to last_page
            params['page'] = i              # set page param to the current crawling page
            response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

            if response.status_code == 200:
                print('request done page', i)
                for record in response.json().get('data',[]): # this to ensure an empty if get method do not get any data inside
                    product_id = record.get('id')
                    product_data.append({'product_id': product_id, 'category_name': category_name, 'max_page': last_page})
            else:
                print('cannot request page',i)
                page_error.append([i,category_id,category_name])  # For those pages cannot be crawled, I put it into page_error to crawl later. We must store these 3 variables to put them in to website params
            time.sleep(random.randrange(3, 10))

        # Export file
        product_df = pd.DataFrame(product_data,columns = ['product_id','category_name','max_page'])    # Make a dataframe of crawled ID, category_name, max_page of that category
        output_file_name = f'GIAY-NAM-{category_name}.csv'  #Export to csv file the IDs in that category. Every sub category has a distinct file. I will finally concat later in another script
        product_df.to_csv(output_file_name)


    

total pages 22
CATEGORY  giay-the-thao-nam-co-cao
request done page 1
request done page 2
request done page 3
request done page 4
request done page 5
request done page 6
request done page 7
request done page 8
request done page 9
request done page 10
request done page 11
request done page 12
request done page 13
request done page 14
request done page 15
request done page 16
request done page 17
request done page 18
request done page 19
request done page 20
request done page 21
request done page 22
total pages 50
CATEGORY  giay-tay-nam-khong-day
total pages 50
CATEGORY  dep-nam-quai-ngang


# Re-Crawl page_error (if any)

In [10]:
if len(page_error) == 0:
    print('There is no error page to crawl again')

max_attempts = 3  # Maximum number of request attempts
for param_set in page_error:
    retry = 0  # Count for tracking the number of retry

    while retry < max_attempts:
        params = {
            'limit': '40',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
            'category': param_set[1],
            'page': param_set[0],
            'urlKey': param_set[2]
        }
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

        if response.status_code == 200:
            print(f'Request done for page {param_set[0]}')
            for record in response.json().get('data'):
                product_id = record.get('id')
                product_data.append({'product_id': product_id, 'category_name': category_name, 'max_page': last_page})
            break  # Break the while loop early

        # If still have error for this turn
        retry += 1

    # Check if the last request attempt could not request any better
    if retry == max_attempts:
        print(f'Reached maximum number of request attempts for the following', param_set)


There is no error page to crawl again


# Crawl ID with 50 pages more

In [ ]:
#Define price range in order to divide category with more than 50 pages into smaller groups
price_range = [
    '500,15000', '15001,30000', '30001,50000', '50001,80000', '80001,150000',
    '150001,250000', '250001,350000', '350001,550000', '550000, 100000000'
                ]

list_page_50 = [] #This list will hold any category that even I have filtered already, it still have page>50. So that I can turn back again and add new price range

for url in list_page_50:
    parsed_url = urlparse(url)
    path_segments = parsed_url.path.split('/')

    category_name = path_segments[1]
    category_id = path_segments[2][1:]
    product_data = [] #This list hold data of products ID, Category, Max_page within that category
    page_error = []  # This list will hold data of error pages that we cannot request. So that we can crawl again

    # The code is similar to the code above, only change 1 param 'price'
    for price in price_range: #Iterate through each price range in the list of price_range
        params = {
            'limit': '40',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
            'category': category_id,
            'page': '1',
            'urlKey': category_name,
            'price': price
        }
        
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

        if response.status_code == 200:
            response_json = response.json()
            last_page = response_json.get('paging', {}).get('last_page')
            print('CATEGORY', category_name, price)
            print('total pages', last_page)

            if last_page == 50:
                list_page_50.append([category_name, price])
                continue
            else:
                for i in range(1, last_page + 1):
                    params['page'] = i
                    response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers,params=params)

                    if response.status_code == 200:
                        print('request done page', i)
                        response_json = response.json()
                        for record in response_json.get('data', []): # this to ensure an empty if get method do not get any data inside
                            product_id = record.get('id')
                            product_data.append({'product_id': product_id, 'category_name': category_name,
                                                 'max_page': last_page})
                    else:
                        page_error.append([category_name,category_id,price,i])
                    time.sleep(random.randrange(3, 10))

    product_df = pd.DataFrame(product_data, columns=['product_id', 'category_name', 'max_page'])
    output_file_name = f'GIAY-NU-50-{category_name}.csv'
    product_df.to_csv(output_file_name, index=False)


# Crawl Again for those error page

In [ ]:
if len(page_error) == 0:
    print('There is no error page to crawl again')

max_attempts = 3  # Maximum number of request attempts
for param_set in page_error:
    retry = 0  # Count for tracking the number of retry

    while retry < max_attempts:
        params = {
            'limit': '40',
            'include': 'advertisement',
            'aggregations': '2',
            'trackity_id': '74634137-8702-8b12-fe42-46a46a2a2573',
            'category': param_set[1],
            'page': param_set[0],
            'urlKey': param_set[2]
        }
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings', headers=headers, params=params)

        if response.status_code == 200:
            print(f'Request done for page {param_set[0]}')
            for record in response.json().get('data'):
                product_id = record.get('id')
                product_data.append({'product_id': product_id, 'category_name': category_name, 'max_page': last_page})
            break  # Break the while loop early

        # If still have error for this turn
        retry += 1

    # Check if the last request attempt could not request any better
    if retry == max_attempts:
        print(f'Reached maximum number of request attempts for the following', param_set)
